In [2]:
import numpy as np
import pandas as pd
import os
from tqdm.auto import tqdm
files = os.listdir("data/미세먼지")
print(files)

['2019년 7월.xlsx', '2019년 5월.xlsx', '2020년 5월.xlsx', '2019년 8월.xlsx', '2020년 1월.xlsx', '2019년 12월.xlsx', '2020년 2월.xlsx', '2019년 3월.xlsx', '2020년 4월.xlsx', '2019년 9월.xlsx', '2019년 2월.xlsx', '2019년 11월.xlsx', '2020년 3월.xlsx', '2019년 10월.xlsx', '2019년 1월.xlsx', '2019년 6월.xlsx', '2019년 4월.xlsx']


In [10]:
def change_region(x):
    regions = {'서울' : '서울특별시',
              '인천' : '인천광역시',
              '부산' : '부산광역시',
              '대구' : '대구광역시',
              '광주' : '광주광역시',
              '대전' : '대전광역시',
              '울산' : '울산광역시',
              '경기' : '경기광역시',
              '강원' : '강원도',
              '충북' : '충청북도',
              '충남' : '충청남도',
              '전북' : '전라북도',
              '전남' : '전라남도',
              '경북' : '경상북도',
              '경남' : '경상남도',
              '제주' : '제주도',
              '세종' : '세종특별자치시'}
    for key, value in regions.items():
        if x == key:
            return value

In [5]:
from datetime import datetime
def convert_time(mytime):
    mytime = str(mytime)
    y = int(mytime[:4])
    m = int(mytime[4:6])
    d = int(mytime[6:8])
    H = int(mytime[8:10]) - 1
    mytime = datetime(y, m, d, H)
    return mytime

In [13]:
def prep_data(data):
    data = data[data['망'] == '도시대기']
    data['지역'] = data['지역'].apply(lambda x: x.split()[0])
    data['측정일시'] = data['측정일시'].astype('U')
    
    data['date'] = data['측정일시'].apply(lambda x: convert_time(x)).apply(lambda x: x.strftime("%Y-%m-%d"))
    data['month'] = data['측정일시'].apply(lambda x: convert_time(x)).apply(lambda x: x.strftime("%Y-%m"))
    data['hour'] = data['측정일시'].apply(lambda x: convert_time(x)).apply(lambda x: x.strftime("%H"))
    data = data.drop(['망', '측정소코드', '측정소명', '측정일시', '주소'], axis = 1)
    
    data = data.groupby(['date', 'month', 'hour','지역']).mean().reset_index()
    data['region'] = data['지역'].apply(change_region)
    
    #interpolate null values
    data = data.interpolate(method='values')
    del data['지역']
    return data

In [16]:
def prep_pop():
    pop = pd.read_csv("population_proportion.csv")
    pop.index = pd.date_range('2019-01', '2020-08', freq='M').strftime('%Y-%m')
    del pop['전국']
    pop = pop.unstack().reset_index()
    pop.columns = ['region', 'month', 'p']
    return pop

def mul(x, df):
    return x*df['p']

In [19]:
pop = prep_pop()

def merged(data):
    data1 = prep_data(data)
    data1 = data1.merge(pop, on = ['region', 'month'])
    return data1

In [21]:
final = pd.DataFrame()
for file in tqdm(files):
    data = pd.read_excel(os.path.join("data/미세먼지", file))
    data1 = merged(data)
#     display(data1.head())
    data1[['SO2', 'CO', 'NO2', 'PM10', 'PM25']] = data1[['SO2', 'CO', 'NO2', 'PM10', 'PM25']].apply(lambda x: mul(x, pop)).values
    del data1['p']
    data1 = data1.groupby(['date', 'month', 'hour']).sum().reset_index()
#     df.head()
    final = pd.concat([final, data1]).reset_index(drop = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [22]:
final

,date,month,hour,SO2,CO,O3,NO2,PM10,PM25
0,2019-07-01,2019-07,00,0.000341,0.075377,0.570114,0.001112,8.937570,6.001902
1,2019-07-01,2019-07,01,0.000341,0.073611,0.554063,0.000980,8.680660,6.283829
2,2019-07-01,2019-07,02,0.000332,0.078988,0.533426,0.000933,8.643773,6.503025
3,2019-07-01,2019-07,03,0.000314,0.073572,0.498991,0.000838,8.541534,6.489891
4,2019-07-01,2019-07,04,0.000341,0.072634,0.446408,0.000876,8.142172,6.157742
...,...,...,...,...,...,...,...,...,...
12403,2019-04-30,2019-04,19,0.000000,0.000000,0.582140,0.000000,0.000000,0.000000
12404,2019-04-30,2019-04,20,0.000000,0.000000,0.519778,0.000000,0.000000,0.000000
12405,2019-04-30,2019-04,21,0.000000,0.000000,0.471819,0.000000,0.000000,0.000000
12406,2019-04-30,2019-04,22,0.000000,0.000000,0.421216,0.000000,0.000000,0.000000


In [24]:
final = final.sort_values(by = ['date', 'month', 'hour']).reset_index(drop = True)
final

,date,month,hour,SO2,CO,O3,NO2,PM10,PM25
0,2019-01-01,2019-01,00,0.000660,0.143216,0.173666,0.004912,8.015098,5.087954
1,2019-01-01,2019-01,01,0.000566,0.137347,0.171010,0.004942,7.567525,5.482640
2,2019-01-01,2019-01,02,0.000579,0.145409,0.185895,0.004968,7.297391,4.900831
3,2019-01-01,2019-01,03,0.000579,0.139966,0.194004,0.004832,6.809900,4.898821
4,2019-01-01,2019-01,04,0.000592,0.129127,0.204225,0.004748,6.375644,4.761557
...,...,...,...,...,...,...,...,...,...
12403,2020-05-31,2020-05,19,0.000000,0.000000,0.905986,0.000000,0.000000,0.000000
12404,2020-05-31,2020-05,20,0.000000,0.000000,0.857956,0.000000,0.000000,0.000000
12405,2020-05-31,2020-05,21,0.000000,0.000000,0.816178,0.000000,0.000000,0.000000
12406,2020-05-31,2020-05,22,0.000000,0.000000,0.784161,0.000000,0.000000,0.000000


In [25]:
len(final[final['O3']==0])

0

In [26]:
final.to_csv("인구가중힙미세먼지.csv", index = False)